In [89]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from pdb import set_trace as br

class ChamferLoss(nn.Module):

	def __init__(self):
		super(ChamferLoss, self).__init__()
		self.eps = 1e-8

	def forward(self,preds,gts):
		P = self.batch_pairwise_dist(gts, preds)
		br()
		mins, _ = torch.min(P, 1)
		loss_1 = torch.sum(mins)
		mins, _ = torch.min(P, 2)
		loss_2 = torch.sum(mins)

		return loss_1 + loss_2


	def batch_pairwise_dist(self,x,y):
		bs, num_points, points_dim = x.size()
		#br()
		xx = torch.bmm(x, x.transpose(2,1))
		yy = torch.bmm(y, y.transpose(2,1))
		zz = torch.bmm(x, y.transpose(2,1))
		diag_ind = torch.arange(0, num_points).type(torch.LongTensor)
		rx = xx[:, diag_ind, diag_ind].unsqueeze(1).expand_as(xx)
		ry = yy[:, diag_ind, diag_ind].unsqueeze(1).expand_as(yy)
		br()
		P = (rx.transpose(2,1) + ry - 2*zz)
		return P

In [92]:
loss = ChamferLoss()
log_input = torch.randn(1, 4, 3)
target = torch.randn(1, 4, 3)
log_input = torch.FloatTensor([[[1,1,1],[2,2,2],[3,3,3]], [[4,4,4],[5,5,5],[6,6,6]]])
target = torch.FloatTensor([[[1,1,1],[2,2,2],[3,3,3]], [[1,1,1],[2,2,2],[3,3,3]]])
input_ = torch.autograd.Variable(log_input, requires_grad=True)
target_ = torch.autograd.Variable(target, requires_grad=False)

In [95]:
output = loss(input_, target_)
output.backward()

> <ipython-input-89-856b9fe55540>(34)batch_pairwise_dist()
-> P = (rx.transpose(2,1) + ry - 2*zz)
(Pdb) continue
> <ipython-input-89-856b9fe55540>(16)forward()
-> mins, _ = torch.min(P, 1)
(Pdb) P
Variable containing:
(0 ,.,.) = 
   0   3  12
   3   0   3
  12   3   0

(1 ,.,.) = 
  27  48  75
  12  27  48
   3  12  27
[torch.FloatTensor of size 2x3x3]

(Pdb) mins, _ = torch.min(P, 1)
(Pdb) loss_1 = torch.sum(mins)
(Pdb) loss_!
*** SyntaxError: invalid syntax (<stdin>, line 1)
(Pdb) loss_1
Variable containing:
 42
[torch.FloatTensor of size 1]

(Pdb) torch.min(P, 1)
(Variable containing:
  0   0   0
  3  12  27
[torch.FloatTensor of size 2x3]
, Variable containing:
 0  1  2
 2  2  2
[torch.LongTensor of size 2x3]
)
(Pdb) mins, _ = torch.min(P, 2)
(Pdb) mins
Variable containing:
  0   0   0
 27  12   3
[torch.FloatTensor of size 2x3]

(Pdb) loss_2 = torch.sum(mins)
(Pdb) loss_2
Variable containing:
 42
[torch.FloatTensor of size 1]

(Pdb) q


BdbQuit: 

In [94]:
input_.grad

Variable containing:
(0 ,.,.) = 
   0   0   0
   0   0   0
   0   0   0

(1 ,.,.) = 
  14  14  14
   4   4   4
   6   6   6
[torch.FloatTensor of size 2x3x3]

In [73]:
x = torch.autograd.Variable(torch.Tensor([3,2]), requires_grad=True)

In [74]:
torch.min(x)

Variable containing:
 2
[torch.FloatTensor of size 1]

In [75]:
f = lambda x: torch.min(x)

In [76]:
y = f(x)

In [77]:
y

Variable containing:
 2
[torch.FloatTensor of size 1]

In [78]:
y.backward()

In [79]:
x.grad

Variable containing:
 0
 1
[torch.FloatTensor of size 2]

In [80]:
diag_ind = torch.arange(0, 10).type(torch.LongTensor)

In [81]:
diag_ind


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
[torch.LongTensor of size 10]